In [1]:
import pickle as pk
import argparse
import pickle as pk
import time
from re import S
import torch
import copy
from tqdm import tqdm
import torch.nn as nn
from pytorch_lightning import seed_everything
from torch.nn import functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

from src.data_utils import *
#from src.model_utils import setupTokenizer
from src.datasethandler import NarrationDataSet,train_data_permutated_path,test_data_path,DataSetLoader
from src.inferenceUtils import PerformanceNarrator
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback,T5Config
from src.modeling_bart import BartNarrationModel
from src.modeling_t5 import T5NarrationModel
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from classification_performance_description_inference import ClassificationPerformanceNarration
trained_model_path = 'TrainModels/earlyfusion/bart-base/'
#"Trained_models/trainednarrators/baseline/bart-base/"

modeltype='earlyfusion'

In [4]:
# instantiate the narrator
narrator = ClassificationPerformanceNarration(model_base = "facebook/bart-base",
                                              model_base_dir= trained_model_path,
                                              
                                              modeltype=modeltype,
                                              max_preamble_len=160,
                                              max_len_trg=185,
                                              length_penalty=3.6,
                                              beam_size=8,
                                              repetition_penalty=1.5,
                                              return_top_beams=1,
                                              random_state=453,
                                              )
                                              
# Set up the inference model
narrator.buildModel()

Model Type: earlyfusion


Some weights of DataNarrationBart were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.relative_attention_for_table.key.weight', 'encoder.relative_attention_for_table.value.weight', 'encoder.relative_attention_for_table.key.bias', 'encoder.relative_attention_for_table.value.bias', 'encoder.relative_attention_for_table.query.bias', 'encoder.relative_attention_for_table.query.weight', 'encoder.relative_attention_for_table.mask', 'encoder.relative_attention_for_table.Er']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# The setup is ready to be used

# Sample of the expected format for the performance report
performance_report = {'Recall': ["19.56%", "LOW"],
                      'F1-score': ["9.06%",
                                   "LOW"],
                      'Accuracy': ["68.16%", "LOW"],
                      'AUC': ["50.16%", "MODERATE"]
                      }
class_labels = ['Not Fraud', 'Fraud']
is_balance = False
narrator.beam_size = 4
generatedTexts = narrator.generateTextualExplanation(performance_report, class_labels,
                                                     is_balance=is_balance)
generatedTexts


Global seed set to 453


'The scores achieved by the model on this classification task are as follows: (a) AUC: 50.16%. (b) Recall: 19.56%. These scores are lower than expected, and judging by them, we can conclude that this model has a lower performance as it is not be able to accurately predict the actual labels of multiple test examples. Furthermore, the Accuracy score is only marginally higher than the dummy model constantly assigning the majority class label "Not Fraud" to any given test case.'

In [9]:
# Example 2

# Sample of the expected format for the performance report
performance_report = {'Precision': ["85.56%", "HIGH"],
                      'Sensitivity': ["79.23%", "HIGH"],
                      'Accuracy': ["87.24%", "HIGH"],
                      'AUC': ["89.78%", "HIGH"]
                      }
class_labels = ['Reject', 'Accept']
is_balance = True
generatedTexts = narrator.generateTextualExplanation(performance_report, class_labels,
                                                     is_balance=is_balance)
generatedTexts


Global seed set to 453


"The model's classification performance on this binary classification task as evaluated based on the Precision, Sensitivity, AUC, and Accuracy are 85.56%, 79.23%, 89.78%, and 87.24%, respectively. These scores support the conclusion that this model will be moderately effective at correctly labeling most test samples drawn from any of the two-class labels. Furthermore, the likelihood of misclassifying any given test example is lower."